In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8631,2024-05-14,Brasil Nbb,20:00,R10 Score Vasco da Gama,Bauru,1.83,1.88,147.5,1.84,1.86,-1.5,2.01,2.01,8d2KKD6t,0.546448,0.531915,0.543478,0.537634,0.078363,1.0,0.6,NaN,NaN,148.216,36.001667,0.242900,138.560,22.697151,0.163807,122.068,171.840,171.00,167.04,0.0,0.0,0.0,0.0,0.019059,0.007644,0.000000,1.90,0.380,2.184211,0.673071,0.9,0.226929,2.14,0.428,2.056075,0.492390,0.5,0.007610
8632,2024-05-14,Eua Nba,21:00,New York Knicks,Indiana Pacers,1.72,2.20,216.5,1.93,1.93,-3.5,2.00,2.70,6kRuDu1b,0.581395,0.454545,0.518135,0.518135,0.035941,0.8,0.6,NaN,NaN,245.336,72.789486,0.296693,157.218,12.163467,0.077367,183.648,230.350,170.66,146.28,0.0,0.0,0.0,0.0,0.173169,0.000000,0.210628,0.04,0.008,90.000000,0.610099,0.5,-0.110099,-2.98,-0.596,-2.013423,0.586684,0.4,-0.186684
8633,2024-05-14,Itália Liga A,15:00,Milano,Trento,1.09,7.70,160.5,1.83,1.93,-13.5,2.00,15.00,YsAP7gjl,0.917431,0.129870,0.546448,0.518135,0.047301,0.4,0.6,NaN,NaN,112.018,36.863409,0.329085,143.804,20.437946,0.142124,91.244,167.328,101.48,168.10,0.0,0.0,0.0,0.0,1.063476,0.037612,1.081457,0.80,0.160,0.562500,0.000000,0.0,0.000000,-5.00,-1.000,-6.700000,0.461948,0.3,-0.161948
8634,2024-05-14,Bulgária Nbl,13:00,Spartak Pleven,Rilski Sportist,2.05,1.70,162.5,1.86,1.84,1.5,1.91,1.56,IBwcS2Qi,0.487805,0.588235,0.537634,0.543478,0.076040,0.6,1.0,NaN,NaN,177.310,54.292306,0.306200,129.132,21.477287,0.166320,139.562,132.496,164.82,155.70,0.0,0.0,0.0,0.0,0.131993,0.007644,0.142644,1.06,0.212,4.952830,0.689372,0.9,0.210628,-0.74,-0.148,-4.729730,0.669764,0.7,0.030236
8635,2024-05-14,Eslováquia Extraliga,13:00,Spisski Rytieri,Levice,1.68,2.10,165.5,1.85,1.85,-2.5,1.89,2.48,dUz1LeuR,0.595238,0.476190,0.540541,0.540541,0.071429,0.6,0.4,NaN,NaN,178.312,27.541988,0.154460,121.332,20.997054,0.173055,123.218,134.876,190.92,156.17,0.0,0.0,0.0,0.0,0.157135,0.000000,0.190935,-1.29,-0.258,-2.635659,0.000000,0.0,0.000000,-0.04,-0.008,-137.500000,0.000000,0.0,0.000000
8636,2024-05-14,Geórgia Superliga,09:30,Kavkasia,Gurjaani Delta,2.10,1.68,164.5,1.83,1.87,-3.5,1.83,2.00,bNIBjEv3,0.476190,0.595238,0.546448,0.534759,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,127.286,13.325280,0.104688,0.000,0.000,167.70,142.76,0.0,0.0,0.0,0.0,0.157135,0.015289,0.062772,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8637,2024-05-14,Geórgia Superliga,12:00,Gau,Kutaisi,2.06,1.69,178.5,1.80,1.86,1.5,1.92,1.55,EHynzgfM,0.485437,0.591716,0.555556,0.537634,0.077153,0.0,0.0,NaN,NaN,213.220,48.961806,0.229630,NaN,NaN,NaN,0.000,0.000,178.54,205.92,0.0,0.0,0.0,0.0,0.139536,0.023184,0.150795,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8638,2024-05-14,Islândia Premier League,16:15,Grindavik,Keflavik,1.28,3.44,185.5,1.86,1.80,-7.5,1.80,6.25,b17XcB0J,0.781250,0.290698,0.537634,0.555556,0.071948,0.4,1.0,NaN,NaN,159.196,38.811220,0.243795,178.858,42.485083,0.237535,170.454,209.460,198.94,199.00,0.0,0.0,0.0,0.0,0.647182,0.023184,0.781770,1.94,0.388,0.721649,0.000000,0.0,0.000000,1.20,0.240,10.166667,0.000000,0.0,0.000000
8639,2024-05-14,Islândia Premier League,17:15,Valur,Njardvik,1.44,2.65,173.5,1.86,1.84,-4.5,1.84,3.70,bH3ywfvp,0.694444,0.377358,0.537634,0.543478,0.071803,0.2,0.6,NaN,NaN,128.822,36.707006,0.284944,162.676,37.371488,0.229730,121.990,172.248,107.44,153.43,0.0,0.0,0.0,0.0,0.418386,0.007644,0.474808,1.95,0.390,1.128205,0.000000,0.0,0.000000,-0.83,-0.166,-9.939759,0.000000,0.0,0.000000
8640,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Brasil Nbb,R10 Score Vasco da Gama,Bauru,147.5,1.84,1.000,Over
1,21:00,Eua Nba,New York Knicks,Indiana Pacers,216.5,1.93,1.000,Over
2,15:00,Itália Liga A,Milano,Trento,160.5,1.83,1.000,Over
3,13:00,Eslováquia Extraliga,Spisski Rytieri,Levice,165.5,1.85,1.000,Over
4,16:15,Islândia Premier League,Grindavik,Keflavik,185.5,1.86,1.000,Over
5,17:15,Islândia Premier League,Valur,Njardvik,173.5,1.86,1.000,Over
6,21:00,Venezuela Superliga,Broncos,Pioneros,149.5,1.85,0.628,Over
